In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

24/06/24 17:20:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
Project = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Project") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Employees = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Employees") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [7]:
Project.createOrReplaceTempView("Project")
Employees.createOrReplaceTempView("Employees")

In [9]:
result = spark.sql('''with avg_wkld as (
select
    team,
    round(avg(workload),2)as avg_workload
from
    Project
inner join
    Employees
on
    Project.employee_id = Employees.employee_id
group by team ),
main_info as (select
    Project.employee_id,
    project_id,
    name,
    workload,
    team
from
    Project
inner join
    Employees
on
    Project.employee_id = Employees.employee_id)
select
    main_info.*
from
    main_info
left join
    avg_wkld
on
    main_info.team = avg_wkld.team
where
    workload>avg_workload''')

In [10]:
result.show()

+-----------+----------+----+--------+----+
|employee_id|project_id|name|workload|team|
+-----------+----------+----+--------+----+
|          2|         1| Ali|      90|   B|
|          4|         2| Doe|      68|   A|
+-----------+----------+----+--------+----+

